<a href="https://colab.research.google.com/github/ACCMouli/chandu/blob/main/lllms/analysis/7_sentiment_transformers%20(1).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Sentiment Analysis — Transformers Pipeline (DistilBERT SST-2)


In [ ]:
# !pip install OpenAI
# !pip install langchain
# !pip install langchain_community
# !pip install Cohere
# !pip install langchain-openai langchain-cohere python-dotenv

In [1]:
!pip install transformers

In [2]:
import os
import warnings
warnings.filterwarnings('ignore')
from google.colab import userdata

In [3]:
# If needed, install:
# %pip install -q pandas transformers

In [4]:
import os
import pandas as pd
from transformers import pipeline



In [5]:
CSV_PATH = "https://raw.githubusercontent.com/giridhar276/genai/refs/heads/main/datasets/Bank_Customer_conversations.csv"

In [6]:
# Create the pipeline (optionally: device=0 if you have a GPU)
sentiment_model = pipeline(
    task="sentiment-analysis",
    model="distilbert-base-uncased-finetuned-sst-2-english"
    # , device=0  # uncomment if you want to force GPU (CUDA)
)

config.json:   0%|          | 0.00/629 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

Device set to use cpu


In [7]:
# Helper: normalize labels to lowercase 'positive'/'negative' for consistency
def normalize_label(label):
    label = (label or '').strip().lower()
    if label.startswith('pos'):
        return 'positive'
    return 'negative'


In [8]:
TEXT_COL = "customer_text"

In [9]:
df = pd.read_csv(CSV_PATH)
df = df[[TEXT_COL]].copy()
df[TEXT_COL] = df[TEXT_COL].astype(str).str.strip()
print('Rows:', len(df))
df.head(3)

Rows: 3


,customer_text
0,Ms. Patel: My card ending 0530 got blocked aft...
1,Ms. Patel: I filed a loan application 27-Aug-2...
2,Mr. Singh: I filed a loan application 30-Aug-2...


In [10]:
def classify_one(text: str):
    res = sentiment_model(text)[0]
    return normalize_label(res['label']), float(res['score'])


In [11]:
labels, scores = [], []
for txt in df[TEXT_COL].tolist():
    lab, sc = classify_one(txt)
    labels.append(lab)
    scores.append(sc)
df['sentiment'] = labels
df['score'] = scores


In [12]:
df.to_csv("using_transformers.csv", index=False)
print(f'Saved:')
df[[TEXT_COL, 'sentiment', 'score']].head(10)

Saved:


,customer_text,sentiment,score
0,Ms. Patel: My card ending 0530 got blocked aft...,negative,0.983887
1,Ms. Patel: I filed a loan application 27-Aug-2...,negative,0.999261
2,Mr. Singh: I filed a loan application 30-Aug-2...,negative,0.998038
